In [1]:
import os 
os.environ['CUDA_VISIBLE_DEVICES']='7'
import keras 
from keras.layers.merge import Grad 
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger


Using TensorFlow backend.


In [2]:
#load the data 
import h5py 
import numpy as np 
data_prefix="/srv/scratch/annashch/deeplearning/gradient_exp/"
train_hdf5=h5py.File(data_prefix+"train_data.hdf5",'r')
train_X=train_hdf5['X']['sequence']
train_Y=train_hdf5['Y']['output']

valid_hdf5=h5py.File(data_prefix+'valid_data.hdf5','r')
valid_X=valid_hdf5['X']['sequence']
valid_Y=valid_hdf5['Y']['output']

test_hdf5=h5py.File(data_prefix+"test_data.hdf5",'r')
test_X=test_hdf5['X']['sequence']
test_Y=test_hdf5['Y']['output']

weights_gata=np.load('weights_gata.npy')
Y_grads_gata=np.load('gradients_gata.npy')
weights_tal=np.load('weights_tal.npy')
Y_grads_tal=np.load('gradients_tal.npy')
weights_gata_valid=np.load('weights_gata_valid.npy')
Y_grads_gata_valid=np.load('gradients_gata_valid.npy')
weights_tal_valid=np.load('weights_tal_valid.npy')
Y_grads_tal_valid=np.load('gradients_tal_valid.npy')
weights_gata_test=np.load('weights_gata_test.npy')
Y_grads_gata_test=np.load('gradients_gata_test.npy')
weights_tal_test=np.load('weights_tal_test.npy')
Y_grads_tal_test=np.load('gradients_tal_test.npy')

In [8]:
#build the model -- without gradient reward (this is our baseline)
input_shape=(200,4)
pool_size=10 
inp = keras.layers.Input(shape=input_shape)
conv = keras.layers.Convolution1D(nb_filter=15, filter_length=15, subsample_length=1)(inp)
relu_post_conv = keras.layers.Activation("relu")(conv)
gap = keras.layers.pooling.GlobalAveragePooling1D()(relu_post_conv)
dense = keras.layers.Dense(2)(gap)
sigmoid_out = keras.layers.Activation("sigmoid")(dense)
model = keras.models.Model(input=inp, output=sigmoid_out)
model.compile(optimizer="adam", loss="binary_crossentropy",metrics=["binary_accuracy"])

/users/annashch/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:5: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(strides=1, filters=15, kernel_size=15)`
/users/annashch/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ac..., inputs=Tensor("in...)`


In [9]:
#train the model 
model_output_path="benchmarkModel"
#checkpointer = ModelCheckpoint(filepath=model_output_path)
earlystopper = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
tensorboard = TensorBoard( write_images=True)
csv_logger = CSVLogger('training.benchmark.log')
model.fit(x=np.asarray(train_X), 
          y=np.asarray(train_Y),
          callbacks=[earlystopper,csv_logger,tensorboard],
          batch_size=250,
          epochs=50,
          validation_data=tuple([valid_X,valid_Y]),
          verbose=1)

Train on 2400 samples, validate on 300 samples
Epoch 1/50
2400/2400 [==============================] - 0s - loss: 0.6833 - binary_accuracy: 0.6200 - val_loss: 0.6700 - val_binary_accuracy: 0.6500
Epoch 2/50
2400/2400 [==============================] - 0s - loss: 0.6567 - binary_accuracy: 0.6679 - val_loss: 0.6514 - val_binary_accuracy: 0.6500
Epoch 3/50
2400/2400 [==============================] - 0s - loss: 0.6399 - binary_accuracy: 0.6679 - val_loss: 0.6452 - val_binary_accuracy: 0.6500
Epoch 4/50
2400/2400 [==============================] - 0s - loss: 0.6346 - binary_accuracy: 0.6679 - val_loss: 0.6466 - val_binary_accuracy: 0.6500
Epoch 5/50
2400/2400 [==============================] - 0s - loss: 0.6341 - binary_accuracy: 0.6679 - val_loss: 0.6470 - val_binary_accuracy: 0.6500
Epoch 6/50
2400/2400 [==============================] - 0s - loss: 0.6337 - binary_accuracy: 0.6679 - val_loss: 0.6457 - val_binary_accuracy: 0.6500
Epoch 7/50
2400/2400 [==============================] - 0s 

In [5]:
#build the model with gradient reward 
reload(keras.layers.merge)
from keras.layers.merge import Grad
input_shape=(200,4)
pool_size=10 
inp = keras.layers.Input(shape=input_shape)
conv = keras.layers.Convolution1D(nb_filter=15, filter_length=15, subsample_length=1)(inp)
relu_post_conv = keras.layers.Activation("relu")(conv)
gap = keras.layers.pooling.GlobalAveragePooling1D()(relu_post_conv)
dense = keras.layers.Dense(2)(gap)
sigmoid_out = keras.layers.Activation("sigmoid")(dense)
grad_layer1 = Grad(task_index=0)([inp, dense])
grad_layer2 = Grad(task_index=1)([inp, dense])
model_grad = keras.models.Model(input=inp, output=[sigmoid_out, grad_layer1,grad_layer2])
model_grad.compile(sample_weight_mode=[None,"temporal","temporal"],
                   optimizer="adam", 
                   loss=["binary_crossentropy",
                         keras.losses.get_positionwise_cosine_1d(pool_size=pool_size),
                         keras.losses.get_positionwise_cosine_1d(pool_size=pool_size)],
                   metrics=["binary_accuracy"])

/users/annashch/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(strides=1, filters=15, kernel_size=15)`
/users/annashch/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=Tensor("in...)`


In [7]:
#train the model 
model_output_path="gradModel"
#checkpointer = ModelCheckpoint(filepath=model_output_path)
earlystopper = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
tensorboard = TensorBoard( write_images=True)
csv_logger = CSVLogger('training.grad.log')
model_grad.fit(x=np.asarray(train_X), 
          y=[np.asarray(train_Y),Y_grads_gata,Y_grads_tal],
               sample_weight=[None,weights_gata,weights_tal],
          callbacks=[csv_logger],
               validation_data=tuple([valid_X,
                                      [np.asarray(valid_Y),Y_grads_gata_valid,Y_grads_tal_valid],
                                      [None,weights_gata_valid,weights_tal_valid]
                                     ]),
          batch_size=250,
          epochs=50,
          verbose=1)

Train on 2400 samples, validate on 300 samples
Epoch 1/50
2400/2400 [==============================] - 0s - loss: -0.1882 - activation_2_loss: 0.6345 - grad_1_loss: -0.4475 - grad_2_loss: -0.3752 - activation_2_binary_accuracy: 0.6679 - grad_1_binary_accuracy: 0.9376 - grad_2_binary_accuracy: 0.9494 - val_loss: -0.1664 - val_activation_2_loss: 0.6458 - val_grad_1_loss: -0.4426 - val_grad_2_loss: -0.3697 - val_activation_2_binary_accuracy: 0.6500 - val_grad_1_binary_accuracy: 0.9378 - val_grad_2_binary_accuracy: 0.9493
Epoch 2/50
2400/2400 [==============================] - 0s - loss: -0.1647 - activation_2_loss: 0.6340 - grad_1_loss: -0.4339 - grad_2_loss: -0.3648 - activation_2_binary_accuracy: 0.6679 - grad_1_binary_accuracy: 0.9376 - grad_2_binary_accuracy: 0.9494 - val_loss: -0.1219 - val_activation_2_loss: 0.6453 - val_grad_1_loss: -0.4106 - val_grad_2_loss: -0.3566 - val_activation_2_binary_accuracy: 0.6500 - val_grad_1_binary_accuracy: 0.9378 - val_grad_2_binary_accuracy: 0.9493